# GPU Check

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for gpu in GPUs:
  print(gpu.name, gpu.memoryTotal)

# Imports

In [ ]:
from search_eval.eval_OneShot import Eval_OS
from search_eval.optimizer.SingleImageDataset import SingleImageDataset
from search_eval.utils.common_utils import *
from search_space.search_space import DARTS_UNet

from nni import trace
import nni.retiarii.strategy as strategy
import nni.retiarii.serializer as serializer

from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader
from nni.retiarii.strategy import DARTS as DartsStrategy

import torch

torch.cuda.empty_cache()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

# Strategy

In [ ]:
# Select the Search Strategy
strategy = DartsStrategy()
# strategy = strategy.DartsStrategy()
# strategy = strategy.ENAS()
# strategy = strategy.GumbelDARTS()
# strategy = strategy.RandomOneShot()

# oneshot

In [ ]:
total_iterations = 1200

resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'
phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{45}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{45}.npy')

# Create the lightning module
module = Eval_OS(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                
                learning_rate=0.01, 
                buffer_size=100,
                patience=1000,
                weight_decay= 5e-7,

                show_every=200,
                report_every=25,
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)
val_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=val_loader)


# Create a Search Space
model_space = DARTS_UNet(depth=4)

# fast_dev_run=False

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=lightning, strategy=strategy)
experiment.run(config)

In [ ]:
# stop experiment and clear cache
experiment.stop()
torch.cuda.empty_cache()

In [ ]:

exported_arch = experiment.export_top_models()

exported_arch


In [ ]:
# Create the lightning module
from search_eval.eval_no_search_SGLD_ES import Eval_SGLD_ES
module = Eval_SGLD_ES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                learning_rate=0.01, 
                buffer_size=100,
                patience=1000,
                weight_decay= 5e-7,

                show_every=200,
                report_every=25,
                )

In [3]:
import torch
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def add_gaussian_noise_with_sigma(img, sigma):
    """
    Add Gaussian noise with a specified standard deviation (sigma) to an image.
    
    Parameters:
        img (torch.Tensor): The input image.
        sigma (float): The standard deviation of the Gaussian noise to be added.
        
    Returns:
        torch.Tensor: The image with added Gaussian noise.
    """
    img_copy = img.clone()
    noise = torch.normal(mean=0., std=sigma, size=img_copy.shape)
    noisy_img = img_copy + noise
    return torch.clamp(noisy_img, 0., 1.)

resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'


phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{45}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{45}.npy')

print(compare_psnr(phantom, phantom_noisy))

print(compare_psnr(phantom, add_gaussian_noise_with_sigma(torch.from_numpy(phantom), 0.09).numpy()))

23.213201732876797
22.94609147527924


/tmp/ipykernel_688472/4266821005.py:29: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  print(compare_psnr(phantom, phantom_noisy))
